<a href="https://colab.research.google.com/github/stepan-ha/Labs/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Алгоритм розрахунку індивідуальних вхідних даних

---

Встановіть значення змінної variant: сума номера групи помноженого на 25 і
порядкового номеру студента в списку групи (групі ПМОм-11 відповідає номер
0, групі ПМІм-11 відповідає номер 1, групі ПМІм-12 відповідає номер 2, групі
ПМІм-13 відповідає номер 3). Далі встановіть set.seed(variant) та згенеруйте
значення змінної redundant як заокруглене до цілого (для заокруглення можна
використати функції floor або round) випадкове число з рівномірного на інтервалі
(номер групи + 5, 25 – номер групи) розподілу (функція runif). Також згенеруйте
значення змінної year як заокруглене до цілого випадкове число з рівномірного
на інтервалі (2006, 2008) розподілу.


In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
from sklearn.metrics import mean_squared_error
from scipy.stats import bootstrap
from statsmodels.formula.api import ols
from statsmodels.tools.eval_measures import mse
import statsmodels.api as sm
import math as math
import random as random


Розрахунок індивідуальних даних
---



In [2]:
group_number = 2

student_number = 5

variant = group_number * 25 + student_number
print(f"Мій номер індивідуального завдання: {variant}")

np.random.seed(variant)

redundant = math.floor(random.uniform(group_number + 5, 25 - group_number))
print(f"Redundant: {redundant}%")


Мій номер індивідуального завдання: 55
Redundant: 13%


# Завдання 1

---

Розглянемо набір даних Boston з бібліотеки MASS. Модифікуйте ці дані наступним
чином: встановивши seed, що дорівнює значенню змінної variant, видаліть redundant % спостережень з допомогою функції sample. Використовуючи модифіковані дані, пристосуйте модель логістичної регресії для передбачення у вибраному районі рівня злочинності більшого чи меншого за середній на основі змінних nox та rad. Оцініть тестову помилку цієї моделі логістичної регресії, використовуючи метод валідаційного набору (використати розбиття 50 на 50, попередньо скинувши seed). Повторіть попередню процедуру три рази, використовуючи нові розбиття вибірки на навчальний та тестовий набори. Прокоментуйте отримані результати. Розгляньте модель логістичної регресії для передбачення у вибраному районі рівня злочинності більшого чи меншого за середній на основі змінних nox, rad та medv. Оцініть тестову помилку для
цієї моделі, використовуючи метод валідаційного набору описаний вище.
Прокоментуйте, чи призводить включення нової змінної до зменшення тестової
помилки.

# Завантаження та необхідна модифікація датасету

---


In [3]:
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
boston = pd.read_csv(url)

np.random.seed(variant)
sample_indices = np.random.choice(boston.index, size=int(len(boston) * (1 - redundant / 100)), replace=False)
boston_reduced = boston.loc[sample_indices]
boston_reduced['high_crime'] = (boston_reduced['crim'] > boston_reduced['crim'].mean()).astype(int)

# Розбиття даних 50/50 і тренування логістичної регресії

In [5]:

for i in range(3):
    np.random.seed(i)
    train, test = train_test_split(boston_reduced, test_size=0.5)
    model = LogisticRegression()
    model.fit(train[['nox', 'rad']], train['high_crime'])
    preds = model.predict(test[['nox', 'rad']])
    error = mean_squared_error(test['high_crime'], preds)
    print(f"Тестова помилка (спроба {i+1}): {error}")

Тестова помилка (спроба 1): 0.013636363636363636
Тестова помилка (спроба 2): 0.013636363636363636
Тестова помилка (спроба 3): 0.00909090909090909


# Додавання змінної medv і повторення моделі

In [4]:
for i in range(3):
    np.random.seed(i)
    train, test = train_test_split(boston_reduced, test_size=0.5)
    model = LogisticRegression()
    model.fit(train[['nox', 'rad', 'medv']], train['high_crime'])
    preds = model.predict(test[['nox', 'rad', 'medv']])
    error = mean_squared_error(test['high_crime'], preds)
    print(f"Тестова помилка з medv (спроба {i+1}): {error}")

Тестова помилка з medv (спроба 1): 0.013636363636363636
Тестова помилка з medv (спроба 2): 0.02727272727272727
Тестова помилка з medv (спроба 3): 0.00909090909090909


#Висновки

Додавання змінної medv не зменшило тестову помилку. у другій спробі було помічено зростання похибки до 0.0273. Це означає, що medv не покращує точність моделі, а можливо навіть навпаки.

# Завдання 2

---

Модифікуйте дані Auto наступним чином: встановивши seed, що дорівнює значенню
змінної variant, видаліть redundant % спостережень з допомогою функції sample. На основі цього набору даних обчисліть оцінку середнього змінної mpg. Оцініть стандартну похибку цієї оцінки. Тепер оцініть стандартну похибку розглянутої вище оцінки середнього за допомогою бутстрапу та порівняйте з попередньо отриманим результатом. Обчисліть оцінку для медіани та десятого процентиля змінної mpg. Оцініть стандартні помилки отриманих оцінок допомогою бутстрапу.

In [9]:

data_auto = pd.read_csv('Auto.csv');

print(data_auto.head())

Auto_new = data_auto.drop(data_auto.sample(frac=redundant / 100, random_state=variant).index)

print(f"Початковий розмір: {data_auto.shape}")
print(f"Новий розмір: {Auto_new.shape}")

mpg_mean = Auto_new['mpg'].mean()
mpg_std = Auto_new['mpg'].std() / np.sqrt(len(Auto_new))
print(f"Середнє значення mpg: {mpg_mean}, стандартна похибка: {mpg_std}")

boot_mean_std = bootstrap((Auto_new['mpg'].values,), np.mean, confidence_level=0.95, random_state=variant)
print(f"Бутстрапована стандартна похибка середнього: {boot_mean_std.standard_error}")

    mpg  cylinders  displacement horsepower  weight  acceleration  year  \
0  18.0          8         307.0        130    3504          12.0    70   
1  15.0          8         350.0        165    3693          11.5    70   
2  18.0          8         318.0        150    3436          11.0    70   
3  16.0          8         304.0        150    3433          12.0    70   
4  17.0          8         302.0        140    3449          10.5    70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino  
Початковий розмір: (397, 9)
Новий розмір: (345, 9)
Середнє значення mpg: 23.578260869565216, стандартна похибка: 0.4191609229282855
Бутстрапована стандартна похибка середнього: 0.4235824390835707


# Генерація змодельованих даних для LOOCV

In [10]:

np.random.seed(variant)
x = np.random.normal(size=100)
y = variant * x - ((redundant * 40) / variant) * x**2 + np.random.normal(size=100)

loo = LeaveOneOut()
errors = []

for degree in range(1, 5):
    poly_features = np.vander(x, degree + 1, increasing=True)
    model = sm.OLS(y, poly_features).fit()

    loocv_error = np.mean([mse(model.predict(poly_features[i, :].reshape(1, -1)), y[i]) for i in range(len(y))])
    errors.append(loocv_error)
    print(f"LOOCV помилка для моделі степеня {degree}: {loocv_error}")

min_error_index = errors.index(min(errors)) + 1
print(f"Модель з найменшою LOOCV помилкою має степінь {min_error_index}")

LOOCV помилка для моделі степеня 1: 204.97351686850328
LOOCV помилка для моделі степеня 2: 1.2962524097079886
LOOCV помилка для моделі степеня 3: 1.2924256795810223
LOOCV помилка для моделі степеня 4: 1.2515946681730414
Модель з найменшою LOOCV помилкою має степінь 4
